In [1]:
#Import necessary libraries
from keras.models import load_model
import cv2
import numpy as np
import tkinter
from tkinter import messagebox
import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
# from email.mime.image import MIMEBase

import pyautogui
import imutils
import os

In [2]:
#Initialize Tkinter
root = tkinter.Tk()
root.withdraw()

#Load trained model
model = load_model('model-013.model')

#Classifier to detect faces
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#Capture video
source=cv2.VideoCapture(0)

# Dictionaries containing details of Wearing Mask and Color of rectangle around face. If wearing mask then color would be 
# green and if not wearing mask then color of rectangle around face would be red
labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [3]:
def send_email():
    
    subject = "Face Mask Detection System Alert"
    body = "One visitor violated the Face Mask Policy. See in the Camera to recognise the user.A Person has been detected without a face mask in the Computer Science Block B. Please Alert the authorities."
    sender_email = "***@gmail.com"
    receiver_email = "***@gmail.com"
    password = "***"

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message["Bcc"] = receiver_email  # Recommended for mass emails
    
        # Add body to email
    message.attach(MIMEText(body, "plain"))

    filename = "without_mask.png"  # In same directory as script

    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
        
        
   

In [4]:
flag = False
while(True):
    

    ret, img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
#         print(label)


#         If label = 1 then it means wearing No Mask and 0 means wearing Mask
        if (label == 1):
#              Throw a Warning Message to tell user to wear a mask if not wearing one. This will stay
#             open and No Access will be given He/She wears the mask

            pyautogui.screenshot("without_mask.png")
            print('Email send')
            messagebox.showwarning("Warning","Detected without mask.Please wear a Face Mask")
            send_email()
            messagebox.showwarning("Warning","Email is sent to an authority.")
            break
    
        else:
            pass
            break
        
    if os.path.exists("without_mask.png"):
        os.remove("without_mask.png")
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

Email send
